## Prerequisites

In order to run this notebook one needs to have pyclesperanto installed !

For details on how to do this check: [pyclesperanto_prototype](https://github.com/clEsperanto/pyclesperanto_prototype)

**Important: Right now this only works if one is using a GPU colab runtime!**

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install pyclesperanto-prototype
  ! pip install pyopencl

In [4]:
# import the required libraries
from czitools.metadata_tools import czi_metadata as czimd
from czitools.utils import misc
from pylibCZIrw import czi as pyczi
from pathlib import Path
import os
import pandas as pd
from skimage import measure, segmentation
from tqdm.contrib.itertools import product
#import pyclesperanto_prototype as cle
import pyclesperanto as cle
from typing import Union
import numpy as np
import requests

In [5]:
def cleseg_voroni_otsu(image: np.ndarray,
                       sigma_spot_detection: int = 5,
                       sigma_outline: int = 1,
                       convert2numpy: bool = True,
                       verbose: bool = False) -> Union[np.ndarray, cle.Image]:

    # based on: https://biapol.github.io/HIP_Introduction_to_Napari_and_image_processing_with_Python_2022/07_Image_segmentation/02_voronoi_otsu_labeling.html

    # transfer the image to the GPU
    image_to_segment = cle.asarray(image)

    # blur the image with a given sigma and detect maxima in the resulting image.
    blurred = cle.gaussian_blur(image_to_segment, sigma_x=sigma_spot_detection, sigma_y=sigma_spot_detection, sigma_z=sigma_spot_detection)

    detected_spots = cle.detect_maxima_box(blurred, radius_x=0, radius_y=0, radius_z=0)

    if verbose:
        number_of_spots = cle.sum_of_all_pixels(detected_spots)
        print("Detected spots", number_of_spots)

    # blur it again with a different sigma and run threshold the image
    blurred2 = cle.gaussian_blur(image_to_segment, sigma_x=sigma_outline, sigma_y=sigma_outline, sigma_z=sigma_outline)
    binary = cle.threshold_otsu(blurred2)

    # take the binary spots and segmentation image, apply a binary_and
    # to exclude spots which were detected in the background area.
    selected_spots = cle.binary_and(binary, detected_spots)

    if verbose:
        number_of_spots = cle.sum_of_all_pixels(selected_spots)
        print("Selected spots", number_of_spots)

    # convert back to numpy array
    labeling = cle.masked_voronoi_labeling(selected_spots, binary)

    if convert2numpy:
        labeling = cle.pull(labeling)

    return labeling

In [6]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [7]:
if IN_COLAB:
    filepath = os.path.join(os.getcwd(), "data/w96_A1+A2.czi")

if not IN_COLAB:
    defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
    filepath = os.path.join(defaultdir, "w96_A1+A2.czi")

print("Selected FilePath: ", filepath)

Selected FilePath:  /datadisk1/Github/czitools/data/w96_A1+A2.czi


In [8]:
if IN_COLAB:
    # see: https://forum.image.sc/t/stackview-pyclesperanto-prototype-demo-on-colab-cant-use-gpu/80145/4?u=sebi06
    cle.select_device("cupy")

if not IN_COLAB:
    # list names of all available OpenCL-devices
    print("Available OpenCL devices:" + str(cle.available_device_names()))

    # select a specific OpenCL / GPU device and see which one was chosen
    # please adapt as needed !
    device = cle.select_device("NVIDIA RTX A3000 Laptop GPU")
    print("Used GPU: ", device)

Available OpenCL devices:['NVIDIA GeForce GTX 1050']
Used GPU:  0x5ef6590e62a0


In [9]:
# define columns names for dataframe for the measure objects
cols = ["WellId", "Well_ColId", "Well_RowId", "S", "T", "Z", "C", "Number"]
objects = pd.DataFrame(columns=cols)
results = pd.DataFrame()

# define the nucleus channel and parameters
chindex = 0
sigma_spot_detection = 5
sigma_outline = 1
minsize = 100  # minimum object size [pixel]
maxsize = 500  # maximum object size [pixel]

In [10]:
# define region properties to be measured and their units
to_measure = ('label',
              'area',
              'centroid',
              'max_intensity',
              'mean_intensity',
              'min_intensity',
              'bbox'
              )

units = ["micron**2", "pixel", "pixel", "cts", "counts", "cts", ]

In [11]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(filepath)

# check if dimensions are None (because they do not exist for that image)
size_c = misc.check_dimsize(mdata.image.SizeC, set2value=1)
size_z = misc.check_dimsize(mdata.image.SizeZ, set2value=1)
size_t = misc.check_dimsize(mdata.image.SizeT, set2value=1)
size_s = misc.check_dimsize(mdata.image.SizeS, set2value=1)

In [12]:
# open the original CZI document to read 2D image planes
with pyczi.open_czi(filepath) as czidoc_r:

    # read 2d array by looping over the planes except for the channel
    for s, t, z in product(range(size_s),
                           range(size_t),
                           range(size_z)):

        # get the current plane indices and store them
        values = {'S': s, 'T': t, 'Z': z, 'C': chindex, 'Number': 0}

        # read 2D plane in case there are (no) scenes
        if mdata.image.SizeS is None:
            image2d = czidoc_r.read(plane={'T': t, 'Z': z, 'C': chindex})[..., 0]
        else:
            image2d = czidoc_r.read(plane={'T': t, 'Z': z, 'C': chindex}, scene=s)[..., 0]

        # do the voroni-otsu segmentation with GPU accelleration
        labels = cleseg_voroni_otsu(image2d,
                                    sigma_spot_detection=sigma_spot_detection,
                                    sigma_outline=sigma_outline,
                                    convert2numpy=True,
                                    verbose=False)

        # clear the border by removing "touching" objects
        labels = segmentation.clear_border(labels)

        # measure the specified parameters store in dataframe
        props = pd.DataFrame(measure.regionprops_table(labels,
                                                       intensity_image=image2d,
                                                       properties=to_measure)).set_index('label')

        # filter objects by size
        props = props[(props['area'] >= minsize) & (props['area'] <= maxsize)]

        # add well information from CZI metadata
        try:
            props['WellId'] = mdata.sample.well_array_names[s]
            props['Well_ColId'] = mdata.sample.well_colID[s]
            props['Well_RowId'] = mdata.sample.well_rowID[s]
        except (IndexError, KeyError) as error:
            print('Error:', error)
            print('Well Information not found. Using S-Index.')
            props['WellId'] = s
            props['Well_ColId'] = s
            props['Well_RowId'] = s

        # add plane indices
        props['S'] = s
        props['T'] = t
        props['Z'] = z
        props['C'] = chindex

        values = {"WellId": props['WellId'],
                  "Well_ColId": props['Well_ColId'],
                  "Well_RowId": props['Well_RowId'],
                  "S": s,
                  "T": t,
                  "Z": z,
                  "C": chindex,
                  "Number": props.shape[0]}

        print('Well:', props['WellId'].iloc[0], ' Objects: ', values['Number'])

        # update dataframe containing the number of objects
        objects = pd.concat([objects, pd.DataFrame(values, index=[0])], ignore_index=True)
        results = pd.concat([results, props], ignore_index=True)

  0%|          | 0/2 [00:00<?, ?it/s]

Well: A1  Objects:  109
Well: A2  Objects:  103


In [13]:
# reorder dataframe with single objects and show some results
new_order = list(results.columns[-7:]) + list(results.columns[:-7])
results = results.reindex(columns=new_order)
results[:5]

,WellId,Well_ColId,Well_RowId,S,T,Z,C,area,centroid-0,centroid-1,max_intensity,mean_intensity,min_intensity,bbox-0,bbox-1,bbox-2,bbox-3
0,A1,1,1,0,0,0,0,333.0,12.609610,206.078078,1476.0,1166.438438,942.0,3,191,23,218
1,A1,1,1,0,0,0,0,294.0,12.278912,1014.095238,1923.0,1322.197279,956.0,4,1004,22,1024
2,A1,1,1,0,0,0,0,379.0,24.686016,911.385224,2020.0,1454.395778,974.0,12,902,39,921
3,A1,1,1,0,0,0,0,279.0,29.842294,255.232975,1387.0,1130.892473,942.0,18,247,42,264
4,A1,1,1,0,0,0,0,334.0,29.074850,420.059880,3714.0,2416.416168,780.0,20,407,39,433
